# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [34]:
!pip install selenium

#chromedriver = chrome

In [35]:
from selenium import webdriver # Controla o navegador
from selenium.webdriver.common.keys import Keys #controlar as teclas
from selenium.webdriver.common.by import By #Pegar elementos/itens da web

#Criar navegador
navegador = webdriver.Chrome()

#Entrar no google e pesquisar por cotação do Dólar
navegador.get("https://google.com")

#pegar a cotação do Dólar
navegador.find_element(
    By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar", Keys.ENTER)
cotacao_dolar = navegador.find_element(
    By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

display(cotacao_dolar)

#pesquisar por cotação do Euro
navegador.get("https://google.com")
navegador.find_element(
    By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação Euro", Keys.ENTER)
cotacao_euro = navegador.find_element(
    By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

display(cotacao_euro)

#pesquisar por cotação do ouro https://www.melhorcambio.com/ouro-hoje
navegador.get("https://www.melhorcambio.com/ouro-hoje")

#pegar cotação de ouro
cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".")

display(cotacao_ouro)

navegador.quit() #fechar



'5.0127559999999995'

'5.5418022680000005'

'321.79'

### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [36]:
#importar a base de dados
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [37]:
#atualizar as cotações na base de dados
coluna_moeda_dolar = tabela["Moeda"] == "Dólar"
#loc[linha,coluna]
tabela.loc[coluna_moeda_dolar , "Cotação"] = float(cotacao_dolar)

coluna_moeda_euro = tabela["Moeda"] == "Euro"
tabela.loc[coluna_moeda_euro , "Cotação"] = float(cotacao_euro)

coluna_moeda_ouro = tabela["Moeda"] == "Ouro"
tabela.loc[coluna_moeda_ouro , "Cotação"] = float(cotacao_ouro)

#atualizar preços de compras na base
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.012756,5012.705872,1.40,7017.788221
1,Carro Renault,4500.00,Euro,5.541802,24938.110206,2.00,49876.220412
2,Notebook Dell,899.99,Dólar,5.012756,4511.430272,1.70,7669.431463
3,IPhone,799.00,Dólar,5.012756,4005.192044,1.70,6808.826475
4,Carro Fiat,3000.00,Euro,5.541802,16625.406804,1.90,31588.272928
5,Celular Xiaomi,480.48,Dólar,5.012756,2408.529003,2.00,4817.058006
6,Joia 20g,20.00,Ouro,321.790000,6435.800000,1.15,7401.170000


### Agora vamos exportar a nova base de preços atualizada

In [38]:
# exportar a base de dados para ter o resultado atualizado
tabela.to_excel("Produtos-atualizado.xlsx", index=False)

In [39]:
tabelaA = pd.read_excel("Produtos-atualizado.xlsx")
display(tabelaA)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.012756,5012.705872,1.40,7017.788221
1,Carro Renault,4500.00,Euro,5.541802,24938.110206,2.00,49876.220412
2,Notebook Dell,899.99,Dólar,5.012756,4511.430272,1.70,7669.431463
3,IPhone,799.00,Dólar,5.012756,4005.192044,1.70,6808.826475
4,Carro Fiat,3000.00,Euro,5.541802,16625.406804,1.90,31588.272928
5,Celular Xiaomi,480.48,Dólar,5.012756,2408.529003,2.00,4817.058006
6,Joia 20g,20.00,Ouro,321.790000,6435.800000,1.15,7401.170000
